In [ ]:
!pip install tensorflow keras


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
!kaggle datasets download -d grassknoted/asl-alphabet

Dataset URL: https://www.kaggle.com/datasets/grassknoted/asl-alphabet
License(s): GPL-2.0
asl-alphabet.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip asl-alphabet.zip

Archive:  asl-alphabet.zip
replace asl_alphabet_test/asl_alphabet_test/A_test.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# !unzip asl-alphabet.zip

import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Define the path to your dataset
dataset_path = 'asl_alphabet_train/asl_alphabet_train'

# Define image dimensions
img_width, img_height = 64, 64

# Create empty lists to store images and labels
images = []
labels = []

# Iterate through each subdirectory (representing each alphabet)
for alphabet_folder in os.listdir(dataset_path):
    alphabet_path = os.path.join(dataset_path, alphabet_folder)

    if os.path.isdir(alphabet_path):  # Check if it's a directory
        for image_file in os.listdir(alphabet_path):
            if image_file.endswith(('.jpg','.jpeg')): #added image extensions for robustness
                try:
                    image_path = os.path.join(alphabet_path, image_file)
                    img = Image.open(image_path).convert('L')  # Open image and convert to grayscale
                    img = img.resize((img_width, img_height))  # Resize the image
                    img_array = np.array(img) / 255.0  # Normalize pixel values
                    images.append(img_array)
                    labels.append(alphabet_folder)
                except Exception as e:
                    print(f"Error processing {image_file}: {e}")

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Reshape the image data (if needed)
images = images.reshape(-1, img_width, img_height, 1)

print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels
labels = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# --- Model Training ---

!pip install tensorflow

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5)) # Dropout for regularization
model.add(Dense(29, activation='softmax')) # Output layer with 29 classes (assuming 29 alphabets)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test)) # Adjust epochs and batch_size as needed

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")